In [1]:
from pathlib import Path
from langchain.agents import initialize_agent, Tool
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
import sqlite3
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from table_relationships import describe_table_relationships
from tbl_col_info import table_info_and_examples
import os
import pandas as pd
import re
import requests

In [2]:
# Load .env file and get GROQ API key
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [3]:
if not api_key:
    print("GROQ_API_KEY not found in .env file.")

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [6]:
# Configure and return SQLite database connection
def configure_db():
    import glob
    conn = sqlite3.connect(":memory:")
    csv_folder = Path.cwd() / "cooked_data_gk"
    for csv_file in glob.glob(str(csv_folder / "*.csv")):
        table_name = Path(csv_file).stem.lower()
        df = pd.read_csv(csv_file)
        df.to_sql(table_name, conn, index=False, if_exists="replace")
    return SQLDatabase.from_uri("sqlite://", engine_args={"creator": lambda: conn})

# Connect to DB
db = configure_db()
print("📄 Tables Loaded:", db.get_table_names())

📄 Tables Loaded: ['order_items', 'orders', 'sub_d_order_items', 'sub_d_orders', 'tbl_distributor_details', 'tbl_price_margin', 'tbl_product_master', 'tbl_sales', 'tbl_scheme']


C:\Users\engma\AppData\Local\Temp\ipykernel_32492\1603749715.py:14: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use :meth:`~get_usable_table_names` instead.
  print("📄 Tables Loaded:", db.get_table_names())


In [9]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal, List


In [11]:
AvailableTables = Literal['order_items',
'orders',
'sub_d_order_items',
'sub_d_orders',
'tbl_distributor_details',
'tbl_price_margin',
'tbl_product_master',
'tbl_sales',
'tbl_scheme']

In [12]:
class OutputTables(BaseModel):
    tables : List[AvailableTables] = Field(description="List of output tables required for sql query, each must be one of the available tables")